In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential, layers


2023-03-30 19:23:17.917094: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
seq_model = Sequential([
    layers.Flatten(input_shape=(28,28)),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
    
])

In a **functional API** layout, we follow the next 3 steps in order to define our model:
 - 1) Input: define input to the model
 - 2) Layers: define a set of interconnected layers on the input
 - 3) Model: define the model using the input and output layers

In [3]:
from tensorflow.keras.layers import Input

In [4]:
input = Input(shape=(28,28))

In [5]:
from tensorflow.keras.layers import Flatten, Dense, Concatenate

In [6]:
x = Flatten()(input)
x = Dense(128, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

In [7]:
from tensorflow.keras.models import Model

In [8]:
func_model = Model(inputs = input, outputs = predictions)

## Branching models

In [9]:
layer1 = Dense(32)
layer2_1 = Dense(32)(layer1)
layer2_2 = Dense(32)(layer1)
layer2_3 = Dense(32)(layer1)
layer2_4 = Dense(32)(layer1)
merge = Concatenate([layer2_1,layer2_2,layer2_3,layer2_4])

TypeError: Inputs to a layer should be tensors. Got '<keras.layers.core.dense.Dense object at 0x167ef8be0>' (of type <class 'keras.layers.core.dense.Dense'>) as input for layer 'dense_5'.

## Creating a multi-output layer

In [33]:
input_layer = Input(shape=(len(train.columns),))
first_dense = Dense(units='128', activation='relu')(input_layer)
second_dense = Dense(units='128',activation='relu')(first_dense)

y1_output = Dense(units='1', name='y1_output')(second_dense)
third_dense = Dense(units='64', activation='relu')(second_dense)
y2_output = Dense(units='1', name='y2_output')(third_dense)


# define the model with the input layer and a list of output layers
model = Model(inputs=input_layer, outputs=[y1_output,y2_output])
print(model.summary())

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 dense_12 (Dense)               (None, 128)          1152        ['input_4[0][0]']                
                                                                                                  
 dense_13 (Dense)               (None, 128)          16512       ['dense_12[0][0]']               
                                                                                                  
 dense_14 (Dense)               (None, 64)           8256        ['dense_13[0][0]']               
                                                                                            

## prepare the dataset

In [26]:
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils.vis_utils import plot_model
import pandas as pd
import pydot
import numpy as np

df = pd.read_excel("/Users/sonic/Documents/playing/tf_advanced_techniques_specialization/ENB2012_data.xlsx")
df = df.sample(frac=1).reset_index(drop=True)

# split the dataset
train, test = train_test_split(df, test_size=0.2)
train_stats = train.describe()

In [29]:
def format_output(data):
    y1 = data.pop('Y1')
    y1 = np.array(y1)
    y2 = data.pop('Y2')
    y2 = np.array(y2)
    return y1, y2

def norm(x):
    return (x - train_stats['mean'])/ train_stats['std']

In [23]:
train_stats

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
count,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000
mean,0.764886,671.315961,318.061075,176.627443,5.238599,3.482085,0.235098,2.832248,22.315694,24.534283
std,0.106914,88.670359,42.983237,45.442208,1.751390,1.117343,0.131475,1.550520,10.070942,9.426254
min,0.620000,514.500000,245.000000,110.250000,3.500000,2.000000,0.000000,0.000000,6.040000,10.900000
25%,0.690000,612.500000,294.000000,122.500000,3.500000,2.250000,0.100000,2.000000,13.000000,15.737500
50%,0.740000,686.000000,318.500000,220.500000,3.500000,3.000000,0.250000,3.000000,18.950000,22.020000
75%,0.820000,735.000000,343.000000,220.500000,7.000000,4.000000,0.400000,4.000000,31.652500,32.997500
max,0.980000,808.500000,416.500000,220.500000,7.000000,5.000000,0.400000,5.000000,43.100000,48.030000


In [24]:
train_stats.pop('Y1')
train_stats.pop('Y2')

count    614.000000
mean      24.534283
std        9.426254
min       10.900000
25%       15.737500
50%       22.020000
75%       32.997500
max       48.030000
Name: Y2, dtype: float64

In [31]:
train_stats = train_stats.transpose()
train_Y = format_output(train)
test_Y = format_output(test)

In [34]:
# specify the optimizer and compile the model with loss functions for both outputs
optimizer = tf.keras.optimizers.SGD(lr = 0.001)
model.compile(optimizer = optimizer,
              loss = {'y1_output': 'mse', 'y2_output': 'mse'},
              metrics = {'y1_output': tf.keras.metrics.RootMeanSquaredError(),
                         'y2_output': tf.keras.metrics.RootMeanSquaredError()})